Cell 1 - Imports

In [ ]:
# What this notebook does:
# - Loads processed IN718 creep data
# - Visualizes per-temperature trends
# - Fits Norton’s law (A, n, Q) or (A, n | Q fixed)
# - Saves calibrated parameters and figures

import os, sys, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append(os.path.abspath("."))  # repo root
from models.creep.norton import R, norton_law, fit_norton


Cell 2 - Load Data

In [ ]:
csv_path = "data/processed/creep_in718.csv"
df = pd.read_csv(csv_path)

# Adjust names if needed:
stress_col = "stress_MPa"
temp_col   = "T_K"
rate_col   = "epsdot_1_per_s"

df = df[[stress_col, temp_col, rate_col]].dropna().rename(
    columns={stress_col:"stress", temp_col:"T", rate_col:"epsdot"}
)
df.head()


Cell 3 - Quick EDA

In [ ]:
plt.figure()
for T, g in df.groupby(np.round(df["T"], 0)):
    plt.loglog(g["stress"], g["epsdot"], "o", label=f"{T:.0f} K")
plt.xlabel("Stress [MPa] (log)")
plt.ylabel("Creep rate [1/s] (log)")
plt.title("IN718 creep — processed data (log–log)")
plt.legend()
plt.show()

single_T = df["T"].nunique() == 1
single_T, df["T"].unique()


Cell 4 - Fit

In [ ]:
stress = df["stress"].to_numpy()
T = df["T"].to_numpy()
epsdot = df["epsdot"].to_numpy()

if single_T:
    Q_fix = 3.0e5  # J/mol; change if you have a literature value
    from scipy.optimize import curve_fit
    def model_fixed_Q(X, A, n): s, t = X; return norton_law(s, t, A, n, Q_fix)
    popt, pcov = curve_fit(model_fixed_Q, (stress, T), epsdot, p0=(1e-7, 5.0), maxfev=20000)
    A_fit, n_fit = popt
    params = (A_fit, n_fit, Q_fix)
else:
    params, pcov = fit_norton(stress, T, epsdot, p0=(1e-7, 5.0, 3.0e5))

A_fit, n_fit, Q_fit = params
A_fit, n_fit, Q_fit


Cell 5 - Diagnostics

In [ ]:
pred = norton_law(stress, T, A_fit, n_fit, Q_fit)
y, yhat = np.log10(epsdot), np.log10(pred)
r2_log = 1 - ((y - yhat)**2).sum() / ((y - y.mean())**2).sum()
print(f"R^2 (log-space) = {r2_log:.4f}")

plt.figure()
plt.loglog(stress, epsdot, "o", label="data")
sgrid = np.linspace(stress.min()*0.9, stress.max()*1.1, 200)
Tplot = np.median(T)
plt.loglog(sgrid, norton_law(sgrid, Tplot, A_fit, n_fit, Q_fit), "-", label=f"fit @ ~{Tplot:.0f} K")
plt.xlabel("Stress [MPa] (log)"); plt.ylabel("Rate [1/s] (log)")
plt.title("Norton fit — log–log"); plt.legend(); plt.show()

plt.figure()
plt.loglog(epsdot, pred, "o"); lims=[min(epsdot.min(), pred.min())*0.8, max(epsdot.max(), pred.max())*1.2]
plt.plot(lims, lims, "--"); plt.xlabel("Measured [1/s] (log)"); plt.ylabel("Predicted [1/s] (log)")
plt.title("Predicted vs Measured"); plt.show()
